In [12]:
import tkinter as tk
import time

MORSE_CODE_DICT = {
    '.-': 'A', '-...': 'B', '-.-.': 'C', '-..': 'D', '.': 'E', '..-.': 'F', '--.': 'G', '....': 'H',
    '..': 'I', '.---': 'J', '-.-': 'K', '.-..': 'L', '--': 'M', '-.': 'N', '---': 'O', '.--.': 'P',
    '--.-': 'Q', '.-.': 'R', '...': 'S', '-': 'T', '..-': 'U', '...-': 'V', '.--': 'W', '-..-': 'X',
    '-.--': 'Y', '--..': 'Z',
    '-----': '0', '.----': '1', '..---': '2', '...--': '3', '....-': '4', '.....': '5', '-....': '6',
    '--...': '7', '---..': '8', '----.': '9'
}

class MorseCodePasswordApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Touch-Based Password System")
        
        self.start_time = 0
        self.morse_code = []  
        self.password = None 
        self.previous_passwords = set() 

        self.current_screen = None

        self.show_main_menu()

    def show_main_menu(self):
        self.clear_screen()
        self.current_screen = "main_menu"
        
        tk.Label(self.root, text="Morse Code Password System", font=("Arial", 18), fg="white", bg="black").pack(pady=10)
        
        tk.Button(self.root, text="Set Password", command=self.show_set_password_screen, bg="darkgray", fg="black").pack(pady=5)
        tk.Button(self.root, text="Unlock Password", command=self.show_unlock_screen, bg="darkgray", fg="black").pack(pady=5)

    def show_set_password_screen(self):
        self.clear_screen()
        self.current_screen = "set_password"
        
        main_frame = tk.Frame(self.root, bg="black")
        main_frame.pack(fill="both", expand=True)

        chart_frame = tk.Frame(main_frame, bg="black")
        chart_frame.pack(side="right", fill="y", padx=10)
        
        tk.Label(chart_frame, text="Morse Code Chart", font=("Arial", 14), fg="yellow", bg="black").pack(pady=5)
        
        for key, value in MORSE_CODE_DICT.items():
            row_frame = tk.Frame(chart_frame, bg="black")
            row_frame.pack(anchor="w")
            tk.Label(row_frame, text=key, font=("Arial", 12), fg="lightgreen", bg="black", width=10, anchor="e").pack(side="left")
            tk.Label(row_frame, text=value, font=("Arial", 12), fg="yellow", bg="black", width=5, anchor="w").pack(side="left")

        content_frame = tk.Frame(main_frame, bg="black")
        content_frame.pack(side="left", fill="both", expand=True, padx=10, pady=10)

        tk.Label(content_frame, text="Set Your Password", font=("Arial", 18), fg="white", bg="black").pack(pady=5)

        tk.Label(content_frame, text="1. Short touch (<0.25s): '.' (dot)\n2. Long touch (>0.25s): '-' (dash)\n3. Press 'Save Password' to finish.",
                 font=("Arial", 12), fg="lightblue", bg="black", justify="left").pack(pady=10)

        self.result_label = tk.Label(content_frame, text="Current Pattern: ", font=("Arial", 16), fg="lightgreen", bg="black")
        self.result_label.pack(pady=5)

        self.action_label = tk.Label(content_frame, text="Press and hold to input your pattern.", font=("Arial", 14), fg="yellow", bg="black")
        self.action_label.pack(pady=5)

        self.canvas = tk.Canvas(content_frame, width=300, height=300, bg="gray")
        self.canvas.pack(pady=10)
        self.canvas.bind("<Button-1>", self.start_timer)
        self.canvas.bind("<ButtonRelease-1>", self.stop_timer)

        tk.Button(content_frame, text="Save Password", command=self.set_password, bg="darkgray", fg="black").pack(pady=5)
        tk.Button(content_frame, text="Delete Last Input", command=self.delete_last_input, bg="darkgray", fg="black").pack(pady=5)
        tk.Button(content_frame, text="Back", command=self.show_main_menu, bg="darkgray", fg="black").pack(pady=5)

    def show_unlock_screen(self):
        self.clear_screen()
        self.current_screen = "unlock_password"
        
        tk.Label(self.root, text="Enter Password to Unlock", font=("Arial", 18), fg="white", bg="black").pack(pady=10)
        
        self.result_label = tk.Label(self.root, text="Current Pattern: ", font=("Arial", 18), fg="lightgreen", bg="black")
        self.result_label.pack(pady=5)
        
        self.action_label = tk.Label(self.root, text="Press and hold to input your pattern.", font=("Arial", 14), fg="yellow", bg="black")
        self.action_label.pack(pady=5)
        
        self.canvas = tk.Canvas(self.root, width=300, height=300, bg="gray")
        self.canvas.pack(pady=10)
        self.canvas.bind("<Button-1>", self.start_timer)
        self.canvas.bind("<ButtonRelease-1>", self.stop_timer)
        
        tk.Button(self.root, text="Check Password", command=self.check_password, bg="darkgray", fg="black").pack(pady=5)
        tk.Button(self.root, text="Delete Last Input", command=self.delete_last_input, bg="darkgray", fg="black").pack(pady=5)
        tk.Button(self.root, text="Back", command=self.show_main_menu, bg="darkgray", fg="black").pack(pady=5)

    def start_timer(self, event):
        self.start_time = time.time()
        self.action_label.config(text="Touch started...", fg="yellow")

    def stop_timer(self, event):
        elapsed_time = time.time() - self.start_time
        if elapsed_time < 0.25:  # 0.25초 미만 -> 점
            self.morse_code.append('.')
            self.action_label.config(text="Short touch recorded (.)", fg="lightblue")
        else:  # 0.25초 이상 -> 대시
            self.morse_code.append('-')
            self.action_label.config(text="Long touch recorded (-)", fg="lightblue")
        self.update_display()

    def update_display(self):
        self.result_label.config(text=f"Current Pattern: {''.join(self.morse_code)}")

    def delete_last_input(self):
        if self.morse_code:
            removed = self.morse_code.pop()
            self.action_label.config(text=f"Removed last input: {removed}", fg="orange")
            self.update_display()
        else:
            self.action_label.config(text="No input to delete.", fg="red")

    def convert_to_morse(self, code):
        return MORSE_CODE_DICT.get(code, "?")

    def set_password(self):
        if self.morse_code:
            morse_string = ''.join(self.morse_code)
            translated_password = self.convert_to_morse(morse_string)
            if translated_password == "?":
                self.result_label.config(text="Invalid Morse Code! Use Morse code patterns only.", fg="red")
                self.action_label.config(text="Please use valid Morse code to set your password.", fg="orange")
                self.morse_code = []
                self.update_display()
                return

            if translated_password in self.previous_passwords:
                self.result_label.config(text="Warning: This password has been used before!", fg="red")
                self.action_label.config(text="Please use a different password for better security.", fg="orange")
            else:
                self.password = translated_password
                self.previous_passwords.add(translated_password)
                self.result_label.config(text=f"Password Saved: {self.password}", fg="lightgreen")
                self.action_label.config(text="Password successfully saved! Return to main menu.", fg="yellow")
                self.morse_code = []
        else:
            self.action_label.config(text="No pattern entered. Try again.", fg="red")

    def check_password(self):
        if not self.password:
            self.action_label.config(text="No password set. Set the password first!", fg="red")
            return

        morse_string = ''.join(self.morse_code)
        translated_password = self.convert_to_morse(morse_string)

        if translated_password == self.password:
            self.result_label.config(text="Password Matched! Access Granted.", fg="lightgreen")
            self.action_label.config(text="Password match successful!", fg="yellow")
        else:
            self.result_label.config(text="Incorrect Password. Try Again.", fg="red")
            self.action_label.config(text="Password mismatch. Please try again.", fg="red")
        self.morse_code = []
        self.update_display()

    def clear_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    root.configure(bg="black")
    app = MorseCodePasswordApp(root)
    root.mainloop()
